In [64]:
import tqdm as notebook_tqdm
from transformers import pipeline, AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
import tensorflow as tf

# Complete pipeline

In [94]:
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Device set to use 0


In [95]:
data = [
    'I hate it!',
    'I love it!',
    'what a shame'
]

In [96]:
results = classifier(data)
print(type(results))
print(results)

<class 'list'>
[{'label': 'NEGATIVE', 'score': 0.9995840191841125}, {'label': 'POSITIVE', 'score': 0.9998781681060791}, {'label': 'NEGATIVE', 'score': 0.9997525811195374}]


# Tokenizer

In [35]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [36]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [37]:
inputs = tokenizer(data, padding=True, truncation=True, return_tensors="tf")

In [41]:
for k, v in inputs.items():
    print(k)
    print(v)
    print(100*'-')

input_ids
tf.Tensor(
[[ 101 1045 5223 2009  999  102]
 [ 101 1045 2293 2009  999  102]
 [ 101 2054 1037 9467  102    0]], shape=(3, 6), dtype=int32)
----------------------------------------------------------------------------------------------------
attention_mask
tf.Tensor(
[[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 0]], shape=(3, 6), dtype=int32)
----------------------------------------------------------------------------------------------------


# Automodel

In [42]:
model = TFAutoModel.from_pretrained(checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [45]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(3, 6, 768)


## Classification model

In [63]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)
print(outputs.logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tf.Tensor(
[[ 4.3221097 -3.462332 ]
 [-4.326873   4.685528 ]
 [ 4.5768113 -3.7275045]], shape=(3, 2), dtype=float32)


In [69]:
predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[9.9958402e-01 4.1598637e-04]
 [1.2187405e-04 9.9987817e-01]
 [9.9975258e-01 2.4738541e-04]], shape=(3, 2), dtype=float32)


In [82]:
predictions_label = tf.math.argmax(predictions, axis=-1)
print(model.config.id2label)

{0: 'NEGATIVE', 1: 'POSITIVE'}


In [90]:
for f, l in zip(data, predictions_label.numpy()):
    print(f, '\t--->', model.config.id2label[l.item()])

I hate it! 	---> NEGATIVE
I love it! 	---> POSITIVE
what a shame 	---> NEGATIVE


# Instantiate a transformer model

In [93]:
bert_model = TFAutoModel.from_pretrained("bert-base-cased")
print(type(bert_model))

gpt_model = TFAutoModel.from_pretrained("gpt2")
print(type(gpt_model))

bart_model = TFAutoModel.from_pretrained("facebook/bart-base")
print(type(bart_model))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

<class 'transformers.models.bert.modeling_tf_bert.TFBertModel'>


c:\Users\lucat\PythonProjects\PRIN\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucat\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
All PyTorch model weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the

<class 'transformers.models.gpt2.modeling_tf_gpt2.TFGPT2Model'>


c:\Users\lucat\PythonProjects\PRIN\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucat\.cache\huggingface\hub\models--facebook--bart-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
All PyTorch model weights were used when initializing TFBartModel.

All the weights of TFBartModel were initi

<class 'transformers.models.bart.modeling_tf_bart.TFBartModel'>


# Tokenizer - Part 2

In [97]:
from transformers import AutoTokenizer

In [102]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [111]:
phrase = "Let's try to tokenize!"
tokens = tokenizer.tokenize(phrase)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
final_inputs = tokenizer.prepare_for_model(input_ids)
print(tokens)
print(input_ids)
print(final_inputs['input_ids'])
print(tokenizer.decode(final_inputs['input_ids']))

['let', "'", 's', 'try', 'to', 'token', '##ize', '!']
[2292, 1005, 1055, 3046, 2000, 19204, 4697, 999]
[101, 2292, 1005, 1055, 3046, 2000, 19204, 4697, 999, 102]
[CLS] let ' s try to tokenize! [SEP]


In [113]:
inputs = tokenizer(phrase)
print(tokenizer.decode(inputs['input_ids']))

[CLS] let ' s try to tokenize! [SEP]


In [120]:
for i, j in inputs.items():
    print(f'{i}:\t{j}')

input_ids:	[101, 2292, 1005, 1055, 3046, 2000, 19204, 4697, 999, 102]
token_type_ids:	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask:	[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
